In [ ]:
pip install colorama

In [2]:
import os
from google import genai
import json
import re
from datetime import datetime
from colorama import Fore, Style

In [3]:
# Set up the Gemini client - ensure you have your API key set in environment variables
api_key = os.environ.get("GEMINI_API_KEY")
if not api_key:
    raise ValueError("Please set the GEMINI_API_KEY environment variable")

client = genai.Client(api_key=api_key)
# Get current datetime
now = datetime.now()
# Format datetime as "YYYY/MM/DD HH:MM"
formatted_datetime = now.strftime("%Y/%m/%d %H:%M")

print(formatted_datetime)

2025/03/08 12:16


In [4]:


def clean_string(response):
    """Clean the response string and convert it to a JSON object"""
    # Step 1: Remove Markdown-like syntax (```json and ```)
    cleaned_string = re.sub(r"```(json)?", "", response).strip()

    # Step 2: Convert to JSON object
    data = json.loads(cleaned_string)

    # Step 3: Handle string `"null"` values
    data = {key: None if value == "null" else value for key, value in data.items()}

    return data

def llm_call(prompt):
    """Call the LLM model with the given prompt"""
    response = client.models.generate_content(
        model='gemini-2.0-flash',
        contents=prompt,
    )
    return response.text

def check_slot(current_datetime, history, slots, user_query): # Augmented
    """Generate a response using Gemini based on the query and retrieved context"""
    # model = genai.GenerativeModel(generation_model)
    
    prompt = f"""
You are an AI assistant, reading the transcript of a conversation between an AI and a human.
From the last line of the conversation, extract all proper named entity(here denoted as slots) that match about booking a flight.
Named entities required for booking a flight include name, origin, destination, departure time.

The output should be returned in the following json format.
{{
    "name": "Define the human name identified from the conversation."
    "origin": "Define origin city identified from the conversation. Define only the city."
    "destination": "Define destination city identified from the conversation. Define only the city."
    "departure_time": "Define departure time identified from the conversation. Format should follow yyyy/mm/dd hh:mi"
}}

If there is no match for each slot, assume null.(e.g., user is simply saying hello or having a brief conversation).

EXAMPLE
Conversation history:
Person #1: I want to book a flight。
AI: "Hi，which city do you want to start your journey?"
Current Slots: {{"name": null, "origin": null, "destination": null, "departure_time": null}}
Last line:
Person #1: Shanghai
Output Slots: {{"name": null, "origin": "Shanghai", "destination": null, "departure_time": null}}
END OF EXAMPLE

EXAMPLE
Current datetime: 2023/04/10 11:20
Conversation history:
Person #1: I want to book a flight to Shanghai.
AI: OK, what time do you want to start?
Current Slots: {{"name": null, "origin": "Shanghai", "destination": null, "departure_time": null}}
Last line:
Person #1: Tomorrow at 8 a.m.
Output Slots: {{"name": null, "origin": "Shanghai", "destination": null, "departure_time": "2023/08/26 08:00"}}
END OF EXAMPLE

Output Slots must be in json format!

Begin!
Current datetime: {current_datetime}
Conversation history (for reference only):
{history}
Current Slots:
{slots}
Last line of conversation (for extraction):
Human: {user_query}

Output Slots:"""
        
    # response = client.models.generate_content(
    #     model='gemini-2.0-flash',
    #     contents=prompt,
    # )
    
    return llm_call(prompt)

def conversational_filling(current_slot):
    prompt = f"""
You are an assistant please take the current slot and write a question to let user fill in any missing information in the JSON
{current_slot}
Your question:"""
    # response = client.models.generate_content(
    #     model='gemini-2.0-flash',
    #     contents=prompt,
    # )
    
    return llm_call(prompt)

In [ ]:
has_none = True
chat_history = ""
current_slot = {"name": None, "origin": None, "destination": None, "departure_time": None}
print(Fore.GREEN + "Please enter where you want to travel, from where and what time and what is your name?" + Style.RESET_ALL)
while has_none:
    # Step 1: Get user input
    user_query = input("Enter the last line of the conversation: ")
    # Step 2: Call the Gemini model to check the slot
    response = check_slot(formatted_datetime, chat_history, current_slot, user_query)
    current_slot = clean_string(response) # Convert from String to Json
    # Step 3: Store chat history
    chat_history += f"Human: {user_query}\n" 
    chat_history += f"AI: {response}\n"
    has_none = any(value is None for value in current_slot.values())
    # Step 4: Explain what to fill
    print(Fore.GREEN + conversational_filling(current_slot) + Style.RESET_ALL)
    # Show sliot and print extra message.
    print(current_slot)
has_none
print(response)

# https://github.com/iMagist486/Chatbot-Slot-Filling/blob/main/chains/prompt.py

Please enter where you want to travel, from where and what time and what is your name?
Okay, to complete your request, I need some more information. Could you please tell me the origin, destination, departure time for Mew?

{'name': 'Mew', 'origin': None, 'destination': None, 'departure_time': None}
Okay, I see you're planning a trip for "นิว". To help me fill in the missing details, could you please tell me: **Where is นิว traveling from, where is นิว going, and what time is นิว departing?**

{'name': 'นิว', 'origin': None, 'destination': None, 'departure_time': None}
What time would you like to depart?

{'name': 'นิว', 'origin': 'กรุงเทพ', 'destination': 'osaka', 'departure_time': None}
Okay, I see you're planning a trip from Bangkok to Osaka! To help me finalize your travel information, could you please tell me: **Do you have any preferred airlines or flight classes you would like to travel with?**

{'name': 'นิว', 'origin': 'กรุงเทพ', 'destination': 'osaka', 'departure_time': '2025